<h2>Construct keyfiles from project directory containing a Base FVS Rx template.</h2>

In [1]:
import os
import glob
from jinja2 import Template
import pandas as pd
import random
import psycopg2

Create a jinja2 template from a Base_Rx.key file.

In [2]:
# read in the base_rx keyfile template using jinja2 templating
with open(os.path.join('Rx_Template','Base_Rx.key'), 'r') as base_keyfile:
    template = Template(base_keyfile.read())
    print('Found Base_Rx.key and created jinja2 template.')

Found Base_Rx.key and created jinja2 template.


A dictionary for holding the items to insert into an FVS keyfile template using jinja2 templating.

In [3]:
inserts = {}

Specify the FVS input and output databases for insertion in the jinja2 template

In [4]:
inserts['FVSIn'] = 'PNWFIADB_FVSIn'
inserts['FVSOut'] = 'PNWFIADB_FVSOut'

Read the contents of each rx*.kcp file in the Rxs directory and store them as values in an `rxs_dict` dictionary.

In [5]:
rxs_dict = {}
# a dictionary storing the silvicultural keywords for each rx
rx_kcps = glob.glob(os.path.join('Rx_Template', 'Rxs', '*.kcp'))
if len(rx_kcps) > 0:
    print('Found the following kcp files in the Rxs subdirectory:')
    for kcp in rx_kcps:
        fname = os.path.split(kcp)[-1]
        print(fname, end='...')
        # read the kcp file
        key = fname.split('/')[-1].split('.')[0] # key for item in inserts dictionary
        with open(kcp, 'r') as item:
            value = item.read()
        # add the contents of the kcp file to the inserts dictionary
        rxs_dict[key] = value
        print(' added to template.')
else: 
    raise FileNotFoundError('No kcp files found in the Rx_Template directory.')

Found the following kcp files in the Rxs subdirectory:
GrowOnly.kcp... added to template.


A function to use for creating keyfiles.

In [6]:
def create_keyfile(standID, variant, rx, offset):
    '''
    Creates a single FVS keyfile based on the jinja2 template.
    '''
    inserts['ID'] = standID
    inserts['rx'] = rxs_dict[rx]
    # FVS slows down outputting to large databases, so we'll divide output among 10 databases
    #inserts['db_num'] = random.randint(1,10) # add a random number, 1-10 for a output database suffix
    
    fname = 'fvs'+variant+'_stand'+str(standID)+'_'+rx+'_off'+str(offset)+'.key'
    path = os.path.abspath('keyfiles_to_run')
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
    with open(os.path.join('keyfiles_to_run',fname),'w') as keyfile:
        keyfile.write(template.render(**inserts))

In [7]:
def create_keyfiles(stands, variants, rxs, offsets=[0], verbose=False):
    '''
    Creates FVS keyfiles for all stands using Base_Rx.key as a template.
    Arguments:
    stands: List of standIDs that keyfiles will be created for. Required.
    variants: List of 2-letter codes of FVS variant for each stand. Required.
    rxs: a list of rx names to build keyfiles for. Required.
    offsets: optional, a list of offsets, used in FVS to delay implementation of a 
        management regime. e.g., [0, 5, 10]. Defaults to a list with no offsets (i.e., [0]).
    '''
    stands_processed = 0
    keyfiles_written = 0
    num_stands = len(stands)
    num_keys = num_stands * len(rxs) * len(offsets)
    print('Creating {:,} keyfiles for {:,} stands.'.format(num_keys, num_stands))

    if not verbose:
        print('Stands processed', end=": ")
    for i in range(len(stands)):
        if verbose:
            print('Creating keyfiles for stand', stands[i], end='... ')
        stand_keyfiles = 0
        for rx in rxs:
            for offset in offsets:
                # run the create_keyfile function
                create_keyfile(standID=stands.iloc[i], variant=variants.iloc[i], rx=rx, offset=offset) 
                keyfiles_written += 1
                stand_keyfiles += 1
        stands_processed += 1
        if verbose:
            print(stand_keyfiles, 'keyfiles written.')
        else:
            if stands_processed % 100 == 0:
                print('{:,}'.format(stands_processed), end='... ')
    print('Done. Created', keyfiles_written, 'keyfiles for', stands_processed, 'stands.')

Identify stands to run.

In [8]:
pg_engine='postgresql://postgres@localhost:5432/PNWFIADB_FVSIn'

# only grab the stands that have DBH increment recorded
# and which have all needed covariates as non nulls

SQL = '''
SELECT fvs_standinit.stand_id, variant
FROM fvs_standinit, fvs_treeinit
WHERE fvs_standinit.stand_id = fvs_treeinit.stand_id AND
aspect IS NOT NULL AND slope IS NOT NULL and dg IS NOT NULL
AND crratio IS NOT NULL AND dbh IS NOT NULL AND species = 202
AND inv_year <= 2015
GROUP BY fvs_standinit.stand_id, variant
'''

# read in the stands from the FVSIn database
stands = pd.read_sql(sql=SQL, con=pg_engine)

In [9]:
#stands = stands.loc[stands.CountPlot == 'N']
stands.head()

,stand_id,variant
0,680013010497,NC
1,680014010497,NC
2,680016010497,NC
3,681249010497,NC
4,681596010497,NC


Create the keyfiles!

In [10]:
%%time
rxs_to_run = ['GrowOnly']
#stands = stands.sample(n=200)
create_keyfiles(stands=stands.stand_id, variants=stands.variant, rxs=rxs_to_run, verbose=False)

Creating 22,972 keyfiles for 22,972 stands.
Stands processed: 100... 200... 300... 400... 500... 600... 700... 800... 900... 1,000... 1,100... 1,200... 1,300... 1,400... 1,500... 1,600... 1,700... 1,800... 1,900... 2,000... 2,100... 2,200... 2,300... 2,400... 2,500... 2,600... 2,700... 2,800... 2,900... 3,000... 3,100... 3,200... 3,300... 3,400... 3,500... 3,600... 3,700... 3,800... 3,900... 4,000... 4,100... 4,200... 4,300... 4,400... 4,500... 4,600... 4,700... 4,800... 4,900... 5,000... 5,100... 5,200... 5,300... 5,400... 5,500... 5,600... 5,700... 5,800... 5,900... 6,000... 6,100... 6,200... 6,300... 6,400... 6,500... 6,600... 6,700... 6,800... 6,900... 7,000... 7,100... 7,200... 7,300... 7,400... 7,500... 7,600... 7,700... 7,800... 7,900... 8,000... 8,100... 8,200... 8,300... 8,400... 8,500... 8,600... 8,700... 8,800... 8,900... 9,000... 9,100... 9,200... 9,300... 9,400... 9,500... 9,600... 9,700... 9,800... 9,900... 10,000... 10,100... 10,200... 10,300... 10,400... 10,500... 10,60